In [1]:
from datetime import datetime
import itertools
import numpy as np
import nltk
import os
import operator
import sys

In [4]:
#download NLTK model data 
nltk.download("book")

[nltk_data] Downloading collection u'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/sarthak/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/sarthak/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/sarthak/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/sarthak/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/sarthak/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/sarthak/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package depen

True

In [2]:
vocabulary_size = 8000 #for little training time
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

corpora_dir = "/home/sarthak/nltk_data/corpora/state_union"

In [14]:
print("Reading Data...")

file_list = []
for root, _ , files in os.walk(corpora_dir):
    for filename in files:
        file_list.append(os.path.join(root, filename))
        
sentences = []

for files in file_list:
    with open(files, 'r') as fin:
        try:
            str_form = fin.read().replace('\n', '')
            sentences.extend(nltk.sent_tokenize(str_form))
        except UnicodeDecodeError:
            
            pass
sentences[:5]

Reading Data...


["PRESIDENT JOHN F. KENNEDY'S ANNUAL ADDRESS TO A JOINT SESSION OF CONGRESS ON THE STATE OF THE UNION This week we begin anew our joint and separate efforts to build the American future.",
 'But, sadly, we build without a man who linked a long past with the present and looked strongly to the future.',
 '"Mister Sam" Rayburn is gone.',
 'Neither this House nor the Nation is the same without him.Members of the Congress, the Constitution makes us not rivals for power but partners for progress.',
 'We are all trustees for the American people, custodians of the American heritage.']

In [17]:
root

'/home/sarthak/nltk_data/corpora/state_union'

In [4]:
sentences = [sentence_start_token + " " + x + " " + sentence_end_token for x in sentences ]

In [5]:
sentences[:5]

["SENTENCE_START PRESIDENT JOHN F. KENNEDY'S ANNUAL ADDRESS TO A JOINT SESSION OF CONGRESS ON THE STATE OF THE UNION This week we begin anew our joint and separate efforts to build the American future. SENTENCE_END",
 'SENTENCE_START But, sadly, we build without a man who linked a long past with the present and looked strongly to the future. SENTENCE_END',
 'SENTENCE_START "Mister Sam" Rayburn is gone. SENTENCE_END',
 'SENTENCE_START Neither this House nor the Nation is the same without him.Members of the Congress, the Constitution makes us not rivals for power but partners for progress. SENTENCE_END',
 'SENTENCE_START We are all trustees for the American people, custodians of the American heritage. SENTENCE_END']

In [6]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

print("Found ", len(word_freq.items()), " unique words tokens.")

('Found ', 18800, ' unique words tokens.')


In [7]:
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)

word_to_index = dict([(w,i) for i, w in enumerate(index_to_word)])

print("Using vocabulary size ", vocabulary_size)
print("The least frequent word in our vocabulary is '",vocab[-1][0],
     "' and appeared ", vocab[-1][1]," times.")
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

('Using vocabulary size ', 8000)
("The least frequent word in our vocabulary is '", 'orbit', "' and appeared ", 2, ' times.')


In [8]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [9]:
x_example, y_example = X_train[10], y_train[10]
print("The 10th sentence input and expected output to every neuron looks like this\n")
print(list(zip([index_to_word[x] for x in x_example], [index_to_word[y] for y in y_example])))

The 10th sentence input and expected output to every neuron looks like this

[('SENTENCE_START', 'For'), ('For', 'if'), ('if', 'we'), ('we', 'can'), ('can', 'not'), ('not', 'fulfill'), ('fulfill', 'our'), ('our', 'own'), ('own', 'ideals'), ('ideals', 'here'), ('here', ','), (',', 'we'), ('we', 'can'), ('can', 'not'), ('not', 'expect'), ('expect', 'others'), ('others', 'to'), ('to', 'accept'), ('accept', 'them'), ('them', '.'), ('.', 'SENTENCE_END')]


In [10]:
x_example

[2,
 232,
 105,
 11,
 30,
 21,
 1103,
 10,
 109,
 909,
 147,
 1,
 11,
 30,
 21,
 889,
 375,
 6,
 1537,
 70,
 4]

In [32]:
def softmax(x):
    """Compute softmanx values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [33]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim=50, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
        
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def calculate_total_loss(self, x, y):
        L = 0
        # For each sentence...
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            # We only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # Add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    def calculate_loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x,y)/N
    
    def sgd_step(self, x, y, learning_rate):
        # Calculate the gradients
        dLdU, dLdV, dLdW = self.bptt(x, y)
        # Change parameters according to gradients and learning rate
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
    
    def bptt(self, x, y):
        T = len(y)
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])              
                dLdU[:,x[bptt_step]] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        # Calculate the gradients using backpropagation. We want to checker if these are correct.
        bptt_gradients = model.bptt(x, y)
        # List of all parameters we want to check.
        model_parameters = ['U', 'V', 'W']
        # Gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # Get the actual parameter value from the mode, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print ("Performing gradient check for parameter ",pname," with size ",  np.prod(parameter.shape))
            # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # Save the original value so we can reset it later
                original_value = parameter[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                parameter[ix] = original_value + h
                gradplus = model.calculate_total_loss([x],[y])
                parameter[ix] = original_value - h
                gradminus = model.calculate_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # Reset parameter to original value
                parameter[ix] = original_value
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate The relative error: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # If the error is to large fail the gradient check
                if relative_error > error_threshold:
                    print ("Gradient Check ERROR: parameter= ",pname," ix=",ix)
                    print ("+h Loss: ", gradplus)
                    print ("-h Loss: ", gradminus)
                    print ("Estimated_gradient: ", estimated_gradient)
                    print ("Backpropagation gradient: ", backprop_gradient)
                    print ("Relative Error: ", relative_error)
                    return 
                it.iternext()
            print ("Gradient check for parameter ",pname," passed.")

In [34]:
np.random.seed(10)
model = RNN(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print (o.shape)
print (o)

(21, 8000)
[[0.00012467 0.00012468 0.00012489 ... 0.00012492 0.00012476 0.00012494]
 [0.00012561 0.00012548 0.00012608 ... 0.00012447 0.00012434 0.00012584]
 [0.00012516 0.00012406 0.00012555 ... 0.00012464 0.00012586 0.00012498]
 ...
 [0.00012538 0.00012611 0.00012477 ... 0.00012485 0.00012504 0.00012545]
 [0.00012498 0.00012543 0.00012539 ... 0.0001251  0.00012555 0.00012441]
 [0.00012511 0.00012539 0.00012535 ... 0.0001253  0.00012383 0.0001246 ]]


In [35]:
# Proving the predict function works
predictions = model.predict(X_train[10])
print (predictions.shape)
# Gibberish output. We're predicting the next words before even training. LOL
print (predictions)

(21,)
[7292 1139 6591  503 4772 4606 7249 7489 1390 5453 7908 3656 3866 4772
 2966 4878 7794 3904 7948 6992 1974]


In [36]:
print( [index_to_word[w] for w in predictions])

['Proclamation', 'knowledge', 'auto', 'aid', 'imposing', 'resourceful', 'shrunk', '5-percent', 'feel', 'filling', 'Father', 'unwise', 'feeling', 'imposing', 'debates', 'toll', 'immeasurable', 'walks', 'animated', 'heretofore', 'alliance']


In [37]:
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: ", np.log(vocabulary_size))
print ("Actual loss: ", model.calculate_loss(X_train[:1000], y_train[:1000]))

('Expected Loss for random predictions: ', 8.987196820661973)
('Actual loss: ', 8.987214494763874)


In [38]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

('Performing gradient check for parameter ', 'U', ' with size ', 1000)


/home/sarthak/anaconda3/envs/opencv/lib/python2.7/site-packages/ipykernel_launcher.py:111: RuntimeWarning: invalid value encountered in double_scalars


('Gradient check for parameter ', 'U', ' passed.')
('Performing gradient check for parameter ', 'V', ' with size ', 1000)
('Gradient check for parameter ', 'V', ' passed.')
('Performing gradient check for parameter ', 'W', ' with size ', 100)
('Gradient check for parameter ', 'W', ' passed.')


In [39]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print (time,": Loss after num_examples_seen=",num_examples_seen," epoch=",epoch,": ",loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to ", learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [ ]:
np.random.seed(10)
model = RNN(vocabulary_size)
train_with_sgd(model, X_train, y_train)

('2019-09-05 13:42:47', ': Loss after num_examples_seen=', 0, ' epoch=', 0, ': ', 8.987193619060955)
('2019-09-05 14:57:10', ': Loss after num_examples_seen=', 66480, ' epoch=', 5, ': ', 5.239609642640895)
('2019-09-05 16:12:06', ': Loss after num_examples_seen=', 132960, ' epoch=', 10, ': ', 5.0479725984299435)
